<a href="https://colab.research.google.com/github/todayuriku/dazaihupuroject/blob/master/%E7%9F%AD%E6%AD%8C%E3%83%BB%E4%BF%B3%E5%8F%A5%E3%83%95%E3%83%A9%E3%82%B0%E4%BB%98%E3%81%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install japanize_matplotlib

In [ ]:
# 必要なパッケージのインポート
import pandas as pd
import google.generativeai as genai
from google.colab import drive, userdata
import time
import random
from tqdm import tqdm
from google.api_core import exceptions, retry

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import statsmodels.api as sm

import matplotlib.pyplot as plt
import japanize_matplotlib

sns.set()

In [ ]:
japanize_matplotlib.japanize()

In [ ]:
import os
from datetime import datetime

In [ ]:
# Google Driveのマウントとパッケージのインストール
drive.mount('/content/drive')
!pip install google-generativeai

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import google.colab
import google.generativeai as genai

In [ ]:
# 行の表示数を設定（Noneを指定すると全ての行を表示）
pd.set_option('display.max_rows', None)

# 列の表示数を設定（Noneを指定すると全ての列を表示）
pd.set_option('display.max_columns', None)

In [ ]:
# API鍵の設定
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# 1. API Keyの確認
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("API Keyが設定されていません")
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"API Keyの設定エラー: {e}")

In [ ]:
# Gemini Proモデルの初期化
# gemini_pro = genai.GenerativeModel("gemini-1.5-flash")
gemini_pro = genai.GenerativeModel("gemini-1.0-pro")

# APIのテスト
prompt = "こんにちは"
response = gemini_pro.generate_content(prompt)
print(response.text)

こんにちは！ご質問やご要望がございましたら、お気軽にお問い合わせください。


In [ ]:
def generate_output_filename(base_path, base_name, extension):
    # 現在の日時を取得
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d_%H%M%S")

    # ファイル名を組み立てる
    filename = f"{base_name}_{date_time}.{extension}"

    # フルパスを返す
    return os.path.join(base_path, filename)

# 使用例
base_path = '/content/drive/MyDrive/太宰府プロジェクト/data/大宰府短歌・俳句前処理後データ.xlsx'
base_name = 'results_with_sentiment'
extension = 'excel'

# **有用・非有用判定**

In [ ]:

input_file_path = "/content/drive/MyDrive/太宰府プロジェクト/data/大宰府短歌・俳句前処理後データ.xlsx"
output_file_path = generate_output_filename(base_path, base_name, extension)

# データの読み込み
df_cleaner_300 = pd.read_excel(input_file_path)

In [ ]:
df_cleaner_300.head()
df_cleaner_300.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   za      3096 non-null   int64  
 1   句       3096 non-null   object 
 2   データ元    3096 non-null   object 
 3   年齢      114 non-null    object 
 4   性別      0 non-null      float64
 5   在住地     2994 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 145.3+ KB


In [ ]:
# df_cleanerをunique_id、item_name、review、review_titleに絞る
df_cleaner_300 = df_cleaner_300[['句', "データ元", "年齢", '在住地']]
df_cleaner_300.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   句       3096 non-null   object
 1   データ元    3096 non-null   object
 2   年齢      114 non-null    object
 3   在住地     2994 non-null   object
dtypes: object(4)
memory usage: 96.9+ KB


In [ ]:
# item_gr_nameに登場する中身を知りたい
# df_cleaner['item_gr_name'].value_counts()

In [ ]:
df_cleaner_300.isnull().sum()

,0
句,0
データ元,0
年齢,2982
在住地,102


In [ ]:
 df1=df_cleaner_300.iloc[0:1499]
 df2=df_cleaner_300.iloc[1500:3000]
 df3=df_cleaner_300.iloc[3001:3096]

In [ ]:
df4 = df_cleaner_300.iloc[0:10]

▽主な重点事業
(1)危機管理の徹底強化
(2)子どもまんなかの施策展開
(3)市民と交流人口、関係人口の相互発展
(4)新しい公共をテーマとした仕組みづくり
(5)歳出入一体改革の推進
(6)その他、市政全般

In [ ]:
@retry.Retry(predicate=retry.if_exception_type(exceptions.ResourceExhausted))
def analyze_review(review_data):
    """レビューの有用性を分析する関数

    Args:
        review_data: レビュータイトルとテキストを含む辞書またはSeries
    Returns:
        str: 分析結果のテキスト
    """
    review_text = f"短歌・俳句: {review_data['句']}"
    prompt = f"""
    以下に入力する短歌または俳句に、(1)危機管理の徹底強化 (2)子どもまんなかの施策展開 (3)市民と交流人口、関係人口の相互発展 (4)新しい公共をテーマとした仕組みづくり (5)歳出入一体改革の推進　のいずれかのアイデア創出につながるものがあれば該当する数字を出力してください。また、この中のものに該当しない場合は0を出力してください。
    {review_text}
    """
    try:
        response = gemini_pro.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Error analyzing review: {e}")
        return "ERROR"

def analyze_reviews_batch(df, start_idx=0, batch_size=10):
    """指定された開始位置からバッチサイズ分のレビューを分析する

    Args:
        df: 分析対象のデータフレーム
        start_idx: 開始インデックス
        batch_size: バッチサイズ
    Returns:
        analyzed_df: 分析結果を含むデータフレーム
        last_idx: 最後に処理したインデックス
    """
    end_idx = min(start_idx + batch_size, len(df))
    batch = df.iloc[start_idx:end_idx].copy()

    results = []
    for _, row in tqdm(batch.iterrows(), total=len(batch), desc=f"Analyzing {start_idx} to {end_idx}"):
        result = analyze_review(row)
        results.append(result)
        time.sleep(3)  # API制限対策

    batch['analysis_result'] = results
    return batch, end_idx

# 進捗を保存しながら分析を実行
def run_analysis_with_checkpoints(input_df, checkpoint_path='review_analysis_checkpoint.csv',
                                batch_size=10, start_from=0):
    """チェックポイントを保存しながらレビュー分析を実行する

    Args:
        input_df: 入力データフレーム
        checkpoint_path: チェックポイントファイルのパス
        batch_size: バッチサイズ
        start_from: 開始インデックス
    """
    all_results = []
    current_idx = start_from

    try:
        while current_idx < len(input_df):
            # バッチ処理の実行
            analyzed_batch, last_idx = analyze_reviews_batch(
                input_df,
                start_idx=current_idx,
                batch_size=batch_size
            )

            # 結果の保存
            all_results.append(analyzed_batch)
            current_results = pd.concat(all_results)
            current_results.to_csv(checkpoint_path)

            print(f"Checkpoint saved at index {last_idx}")
            current_idx = last_idx

            time.sleep(10)  # バッチ間の待機

    except Exception as e:
        print(f"Error occurred at index {current_idx}: {e}")
        print("You can resume from this index later")

    finally:
        # 最終結果の結合
        final_results = pd.concat(all_results) if all_results else pd.DataFrame()
        return final_results

In [ ]:
results_df1 = run_analysis_with_checkpoints(
    df1,
    checkpoint_path='review_analysis_checkpoint.csv',
    batch_size=10
)

Analyzing 0 to 10: 100%|██████████| 10/10 [00:41<00:00,  4.15s/it]


Checkpoint saved at index 10


Analyzing 10 to 20: 100%|██████████| 10/10 [00:43<00:00,  4.34s/it]


Checkpoint saved at index 20


Analyzing 20 to 30:  80%|████████  | 8/10 [00:32<00:08,  4.00s/it]ERROR:tornado.access:500 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1064.02ms


...

Error analyzing review: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Analyzing 1490 to 1499:  89%|████████▉ | 8/9 [00:26<00:03,  3.35s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 330.65ms


Error analyzing review: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Analyzing 1490 to 1499: 100%|██████████| 9/9 [00:30<00:00,  3.35s/it]


Checkpoint saved at index 1499


In [ ]:
results_df2 = run_analysis_with_checkpoints(
    df2,
    checkpoint_path='review_analysis_checkpoint.csv',
    batch_size=10
)

Analyzing 0 to 10:   0%|          | 0/10 [00:00<?, ?it/s]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 583.33ms


Error analyzing review: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Analyzing 0 to 10:  20%|██        | 2/10 [00:08<00:34,  4.28s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 356.46ms


Error analyzing review: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Analyzing 0 to 10:  40%|████      | 4/10 [00:16<00:25,  4.25s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 381.04ms


...

Analyzing 20 to 30:  60%|██████    | 6/10 [00:20<00:13,  3.38s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 357.14ms


Error analyzing review: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Analyzing 20 to 30:  70%|███████   | 7/10 [00:23<00:10,  3.38s/it]WARNING:tornado.access:429 POST /v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 356.91ms


Error analyzing review: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Analyzing 20 to 30:  70%|███████   | 7/10 [00:25<00:11,  3.69s/it]


In [ ]:
results_df3 = run_analysis_with_checkpoints(
    df3,
    checkpoint_path='review_analysis_checkpoint.csv',
    batch_size=10
)

Analyzing 0 to 10:   0%|          | 0/10 [00:00<?, ?it/s]

Error analyzing review: name 'gemini_pro' is not defined


Analyzing 0 to 10:  10%|█         | 1/10 [00:03<00:27,  3.00s/it]

Error analyzing review: name 'gemini_pro' is not defined


Analyzing 0 to 10:  20%|██        | 2/10 [00:06<00:24,  3.00s/it]

...

Error analyzing review: name 'gemini_pro' is not defined


Analyzing 0 to 10:  90%|█████████ | 9/10 [00:27<00:03,  3.00s/it]

Error analyzing review: name 'gemini_pro' is not defined


Analyzing 0 to 10: 100%|██████████| 10/10 [00:30<00:00,  3.00s/it]


Checkpoint saved at index 10


In [ ]:
results_df1["analysis_result"] = results_df1["analysis_result"].replace({
            '(2)':'2',
            '(2) 子どもまんなかの施策展開':'2',
            '(2)子どもまんなかの施策展開':'2',
            '該当なし: 0':'0',
            '(1)':'1',
            '(3)市民と交流人口、関係人口の相互発展':'3',
             '該当するアイデアはありません。\n0':'0',
             '該当するアイデア創出につながるものはありません。\n0':'0',
             '該当なし (0)':'0'
}, regex=False)

In [ ]:
results_df1["analysis_result"].value_counts()

,count
analysis_result,
0,1401
2,36
ERROR,33
5,10
1,9
3,7
4,3


In [ ]:
results_df1.to_excel("/content/drive/MyDrive/太宰府プロジェクト/data/0_1499.xlsx")

In [ ]:
results_df2.to_excel("/content/drive/MyDrive/太宰府プロジェクト/data/1500_3000.xlsx のコピー")

In [ ]:
results3_df3.to_excel("/content/drive/MyDrive/太宰府プロジェクト/data/3001_3096.xlsx のコピー のコピー")